In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"
branch = "pose_rgb"

if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url}")

if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

%cd 6D_pose

In [ ]:
from google.colab import drive
from utils.load_data import mount_drive

mount_drive()

dataset_root = "/content/drive/MyDrive/Linemod_preprocessed"
print(f"\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

In [ ]:
from src.pose_rgb.pointcloud_dataset import LineModPointCloudDataset
from src.pose_rgb.pointnet_model import PointNetPose
from src.pose_rgb.loss import AutomaticWeightedLoss
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔥 Using device: {DEVICE}")

## 📝 Note sulla Loss Function

**Differenza tra PointNet e RGB approach:**

- **RGB (ResNet + TranslationNet)**: 
  - Predice `[dx, dy, log(z)]` → poi pinhole projection → `[X, Y, Z]`
  - Usa `DisentangledTranslationLoss`: separa XY da Z
  - Ha senso perché XY dipendono dalla geometria pinhole, Z è indipendente

- **PointNet**:
  - Predice **direttamente** `[X, Y, Z]` dalla point cloud
  - Usa loss **unificata** per translation: tratta X, Y, Z simmetricamente
  - Non c'è pinhole projection, quindi non ha senso separare XY da Z

## ⚡ Performance Optimizations

**Ottimizzazioni applicate per velocizzare il training:**

1. **Riduzione punti**: 1024 → 512 punti per point cloud (~2x speed-up)
2. **Cached intrinsics**: Usa `linemod_config` invece di caricare YAML ogni volta (~2x speed-up)
3. **Torch sampling**: `torch.randperm` invece di `np.random.choice` (~1.5x speed-up)
4. **Mixed precision**: FP16/FP32 automatico con `torch.cuda.amp` (~1.5x speed-up)
5. **Batch size**: Aumentato da 32 → 64 per migliore GPU utilization
6. **DataLoader**: `num_workers=4` + `pin_memory=True` per I/O parallelo

**Speed-up totale stimato: ~5-8x** rispetto alla versione iniziale! 🚀

In [ ]:
# Crea dataset con point clouds
# use_rgb=True -> point cloud con 6 canali [x,y,z,r,g,b]
# use_rgb=False -> point cloud con 3 canali [x,y,z]

train_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='train',
    num_points=512,  # Ridotto per speed-up (era 1024)
    use_rgb=True      # Include RGB
)

test_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='test',
    num_points=512,
    use_rgb=True
)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Visualizza un sample
sample = train_dataset[0]

print("Sample keys:", sample.keys())
print(f"Point cloud shape: {sample['point_cloud'].shape}")  # (1024, 6)
print(f"Rotation shape: {sample['rotation'].shape}")        # (4,)
print(f"Translation shape: {sample['translation'].shape}")  # (3,)
print(f"\nRotation (quat): {sample['rotation']}")
print(f"Translation (m): {sample['translation']}")

In [ ]:
# DataLoaders - Ottimizzati per performance
train_loader = DataLoader(
    train_dataset, 
    batch_size=64,  # Aumentato da 32 per migliore GPU utilization
    shuffle=True, 
    num_workers=4,  # Aumentato da 2
    pin_memory=True  # Velocizza transfer CPU->GPU
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [ ]:
import os
import json
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler

# ==========================================
# HYPERPARAMETERS
# ==========================================
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
USE_MIXED_PRECISION = True  # Mixed precision per speed-up ~1.5x

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
CHECKPOINT_DIR = f'/content/drive/MyDrive/runs/pointnet_{timestamp}'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Initialize PointNet Model
# input_channels=6 perché usiamo [x,y,z,r,g,b]
model = PointNetPose(input_channels=6, use_batch_norm=True).to(DEVICE)

# Loss per PointNet: use_disentangled=False perché non usiamo pinhole projection
# Translation viene predetta direttamente come [X,Y,Z] simmetrico
criterion = AutomaticWeightedLoss(use_disentangled=False).to(DEVICE)

# Optimizer
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(criterion.parameters()),
    lr=LEARNING_RATE
)

# Mixed Precision Scaler
scaler = GradScaler(DEVICE) if USE_MIXED_PRECISION else None

train_losses = []
val_losses = []
best_val_loss = float('inf')
best_epoch = 0

print(f"\n🔥 STARTING POINTNET TRAINING on {DEVICE}...")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"⚙️  Loss mode: Unified Translation (no XY/Z separation)")
print(f"⚡ Mixed Precision: {USE_MIXED_PRECISION}")

In [ ]:
# ==========================================
# TRAINING LOOP (con Mixed Precision)
# ==========================================
for epoch in range(NUM_EPOCHS):
    
    # --- TRAIN PHASE ---
    model.train()
    running_train_loss = 0.0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    
    for batch in pbar:
        # Move to device
        point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)  # (B, N, 6)
        gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)           # (B, 4)
        gt_trans = batch['translation'].to(DEVICE, non_blocking=True)      # (B, 3) in metri
        
        optimizer.zero_grad()
        
        # Mixed Precision Forward + Backward
        if USE_MIXED_PRECISION:
            with autocast(DEVICE):
                pred_rot, pred_trans = model(point_clouds)
                loss, l_r, l_t, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            pred_rot, pred_trans = model(point_clouds)
            loss, l_r, l_t, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            loss.backward()
            optimizer.step()
        
        running_train_loss += loss.item()
        
        pbar.set_postfix({
            'L_Tot': f"{loss.item():.2f}",
            'L_Rot': f"{l_r.item():.2f}",
            'L_Trans': f"{l_t.item():.3f}"
        })
    
    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # --- VALIDATION PHASE ---
    model.eval()
    running_val_loss = 0.0
    val_batches_limit = 50
    count_batches = 0
    
    with torch.no_grad():
        val_iterator = islice(test_loader, val_batches_limit)
        val_pbar = tqdm(val_iterator, total=val_batches_limit, desc="Validating")
        
        for batch in val_pbar:
            point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)
            gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)
            gt_trans = batch['translation'].to(DEVICE, non_blocking=True)
            
            if USE_MIXED_PRECISION:
                with autocast(DEVICE):
                    pred_rot, pred_trans = model(point_clouds)
                    loss, _, _, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            else:
                pred_rot, pred_trans = model(point_clouds)
                loss, _, _, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            
            running_val_loss += loss.item()
            count_batches += 1
    
    avg_val_loss = running_val_loss / count_batches if count_batches > 0 else 0.0
    val_losses.append(avg_val_loss)
    
    # --- REPORT & SAVE ---
    print(f"📊 Epoch {epoch+1}: Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")
    
    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch + 1
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
            'val_loss': best_val_loss
        }, os.path.join(CHECKPOINT_DIR, "best_model.pth"))
        
        print(f"🏆 New Best Model! (Loss: {best_val_loss:.4f})")
    
    # Save last checkpoint
    if (epoch + 1) == NUM_EPOCHS:
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
        }, os.path.join(CHECKPOINT_DIR, f"checkpoint_ep{epoch+1}.pth"))

print("\n🎉 TRAINING COMPLETE!")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5, 
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PointNet Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5, 
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.yscale('log')
plt.title('PointNet Training History (Log)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'training_history.png'), dpi=150)
plt.show()

print(f"\n📊 Training Statistics:")
print(f"   Best epoch: {best_epoch}")
print(f"   Best val loss: {best_val_loss:.6f}")
print(f"   Final train loss: {train_losses[-1]:.6f}")

# Save history
history = {
    'train_losses': [float(x) for x in train_losses],
    'val_losses': [float(x) for x in val_losses],
    'best_epoch': int(best_epoch),
    'best_val_loss': float(best_val_loss),
    'timestamp': timestamp
}

with open(os.path.join(CHECKPOINT_DIR, 'history.json'), 'w') as f:
    json.dump(history, f, indent=2)

## Visualize Point Cloud Sample

Visualizziamo una point cloud dal dataset per verificare che tutto funzioni correttamente.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Visualizza una point cloud
sample = train_dataset[100]
pc = sample['point_cloud'].numpy()  # (1024, 6) [x, y, z, r, g, b]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot 3D points con colori RGB
ax.scatter(pc[:, 0], pc[:, 1], pc[:, 2], 
           c=pc[:, 3:6], s=2, alpha=0.6)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title(f'Point Cloud - Object {sample["object_id"]}')
plt.show()

print(f"Object ID: {sample['object_id']}")
print(f"Rotation (quat): {sample['rotation']}")
print(f"Translation (m): {sample['translation']}")